## Mở đầu.
- `Classification problem` hay được biết đến như bài toán dự đoán việc `phân nhóm` dữ liệu, là một trong nhiều bài toán thường gặp của `Machine Learning`, nó có thể được ứng dụng trong nhiều lĩnh vực khác nhau của cuộc sống như `dự đoán khả năng mắc bệnh dựa trên các quan trắc y khoa`; `phân loại loài hoa dựa trên các quan trắc sinh học như độ dài cành hoa, đài hoa, cuống hoa, ...`
- Có rất nhiều phương pháp (`method`), công cụ (`tool, metric`) khác nhau để giải quyết bài toán phân loại (`classification problem`) như `Logistic Regression`, `Support Vector Machine (SVM)`, `K-nearest-neighbor (KNN)`, `Naive-Bayes`, `XGBoost`, ...
- Các phương pháp trên đều cung cấp cho ta những công cụ thích hợp để đánh giá `độ chính xác (accuraccy)` cũng như sai số ước lượng `errors` trong mỗi bài toán.

## Import the basic libraries.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/pima-indians-diabetes-database/diabetes.csv
/kaggle/input/mushroom-classification/mushrooms.csv
/kaggle/input/xAPI-Edu-Data/xAPI-Edu-Data.csv


## Problem 1: Predict the onset of diabetes based on diagnostic measures.
- Cột `outcome` là `target`
- Các cột còn lại là các biến độc lập (`independent variable`) trong dữ liệu để dự đoán

Đầu tiên, tải file và nhìn vào 5 dòng đầu tiên trong dữ liêuj này
### Step 1. Loading & viewing dataset

In [2]:
diabetes = pd.read_csv(r"/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Step 2. EDA. Explotary data analysis
Trước khi đi vào vấn đề dự đoán của bài toán này, chúng ta cần nắm rõ cấu trúc từng loại dữ liệu trong bài toán. Nhìn vào dữ liệu trên, ta có thể thấy rằng
- Cột `Pregnancies : Number of times pregnant` (integers) tức là số lần mang thai của bệnh nhân.
- Cột `Glucose`: (`int`) là nồng độ đường huyết của bệnh nhân.
- Cột `BloodPressure` (`int`) huyết áp
- `Skin Thickness` (`int`) độ dày nếp gắp của da (lượng mỡ dưới da)
- `Insulin` (`int`) Insulin lấy từ huyết thanh.
- `BMI` (`float`) chỉ số khối lượng cơ thể, tính bởi trọng lượng chia bình phương chiều cao.
- `DiabetesPedigreeFunction` (`float`) Kết quả tính toán sự liên hệ giữa bệnh tiểu đường với phả hệ.
- `Age` (`int`) tuổi của bệnh nhân
- `Outcome` (`int`) chỉ chứa 2 giá trị `0` và `1` là bệnh nhân đó có mắc bệnh hay không?

**Step 2.1. Checking `missing-value`**

In [3]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Như vậy, dữ liệu có 768 quan trắc và tất cả các cột đều là `non-null` chứng tỏ chúng ta không cần xử lý `missing-data` ở đây!

**Step 2.2. Assign `X` and `y`**.
Sau khi đã biết đâu là `target` và không có dữ liệu khuyết (`missing-data`) thì bấy giờ ta phải gán `X` và `y` vào dataframe cụ thể trước khi sang bước kế tiếp : `train-test-split`

In [4]:
X = diabetes.iloc[:, :-1]   ## X là dữ liệu bỏ đi cột cuối cùng Outcome và lấy tất cả các dòng trong table
y = diabetes.iloc[:, -1].to_numpy()     ## y
print(y[:5])
X.head()

[1 0 1 0 1]


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


**Step 2.3. Group by** Để thực hiện phân chia trainning-testing set được tối ưu cho kiểm định độ chính xác, chúng ta cần phải biết liệu rằng `target (y)` ở đây là cột `Outcome` có đồng đều [`50% là 1; 50% là 0`] hay bị lệch về một nhóm? 

In [5]:
diabetes.groupby(['Outcome']).count()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,500,500,500,500,500,500,500,500
1,268,268,268,268,268,268,268,268


Điều này nói lên rằng trong `768` bệnh nhân được khảo sát thì có đến `500` cho kết quả `0` tức là gần gấp đôi số người mắc bệnh với `outcome = 1` trong tổng thể! Khi thực hiện train-test-split thì ta vẫn phải bảo toàn tỷ lệ này trong cả 2 tập đó!

**Step 2.4. Train-test-split** Để đảm bảo điều đó, ta chỉ cần thêm khai báo `stratify = y` vào hàm `train_test_split()`

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    stratify = y, random_state = 0)

Sau đây, ta sẽ kiểm tra lại tỷ lệ `Outcome = 0` và `Outcome = 1` trong các tập `train-test-set` có được bảo toàn từ dữ liệu ban đầu hay không?

In [7]:
print("Tỷ lệ outcome = 1 / outcome = 0 trong y_train là ",(y_train == 1).sum() / (y_train == 0).sum())
print("Tỷ lệ outcome = 1 / outcome = 0 trong y_test là ", (y_test == 1).sum() / (y_test == 0).sum())
print("Tỷ lệ outcome = 1 / outcome = 0 trong tổng thể là ", (y == 1).sum() / (y == 0).sum())

Tỷ lệ outcome = 1 / outcome = 0 trong y_train là  0.5342857142857143
Tỷ lệ outcome = 1 / outcome = 0 trong y_test là  0.54
Tỷ lệ outcome = 1 / outcome = 0 trong tổng thể là  0.536


Do đó, việc phân hoạch `train-test-set` này đã bảo toàn được tỷ lệ các outcome từ tổng thể, bây giờ ta sẽ tiến hành kiểm định

### Step 3. Chọn mô hình dự đoán
Ở đây, để thuận tiện thì ta chỉ dùng mô hình `support vector machine` cho đơn giản nhất, đây chắc chắn chưa phải model tốt nhất nhưng đủ để minh họa cách tìm và tính toán các mục tiêu như `training_accuraccy`, `testing_accuraccy`

In [8]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

clf_svm = SVC(kernel = 'poly', C = 10)   
clf_svm.fit(X_train, y_train)
print("train acc: ", accuracy_score(y_train, clf_svm.predict(X_train)))
print("train acc: ", accuracy_score(y_test, clf_svm.predict(X_test)))

train acc:  0.7839851024208566
train acc:  0.7748917748917749


Sau cùng, nhìn vào `confusion matrix` dưới đây để đưa ra kết quả nhận xét

In [9]:
from sklearn.metrics import classification_report, confusion_matrix
print('Confusion matrix : \n', confusion_matrix(y_test, clf_svm.predict(X_test)))
print('Classification report : \n', classification_report(y_test, clf_svm.predict(X_test), digits = 5)) 

Confusion matrix : 
 [[134  16]
 [ 36  45]]
Classification report : 
               precision    recall  f1-score   support

           0    0.78824   0.89333   0.83750       150
           1    0.73770   0.55556   0.63380        81

    accuracy                        0.77489       231
   macro avg    0.76297   0.72444   0.73565       231
weighted avg    0.77052   0.77489   0.76607       231



**Comments.**
- Từ `confusion matrix` chỉ ra rằng trong tổng số `231` dữ liệu của `test-set` thì có:

$\qquad \qquad \diamond$ `134` kết quả dự đoán chính xác `Outcome = 0` với thực tế `outcome = 0` trong tổng số `150 (outcome = 0)` của `y_test`; tức là **chỉ có** 16 giá trị bị dự đoán sai bởi mô hình `SVM` này khi dự đoán `outcome = 1` nhưng thực tế lại là `outcome = 0`.

$\qquad \qquad \diamond$ `36` kết quả dự đoán chính xác `Outcome = 1` với thực tế `outcome = 1` trong tổng số `81 (outcome = 1)` của `y_test`; tức là **có đến** 45 giá trị bị dự đoán sai bởi mô hình `SVM` này khi dự đoán `outcome = 0` nhưng thực tế lại là `outcome = 1`.

- Từ `confusion matrix` này dẫn đến việc tính toán accuracy cho bài toán này ở đây có độ chính xác là `77%`; chủ yếu vì dự đoán sai nhóm `outcome = 1`. Vấn đề này có thể được hiểu bởi chúng ta chưa thực hiện tốt khâu `EDA` ở bước 2 khi còn giữ lại một số thông tin không quan trọng.

## Problem 2. Mushroom Classification
Cũng bằng lập luận tương tự như khi làm trong Problem 1 ở trước đó, ta vẫn sẽ dùng `SVM` cho bài toán này.

Firstly, loading & viewing your database

In [10]:
mushroom = pd.read_csv(r"/kaggle/input/mushroom-classification/mushrooms.csv")
mushroom.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


Kiểm tra và in ra các giá trị duy nhất ở từng cột.

In [11]:
column_names = mushroom.columns
for col in column_names:
    print("Các giá trị duy nhất trong cột %s là:\n\t %s"%(col, mushroom[col].unique()))

Các giá trị duy nhất trong cột class là:
	 ['p' 'e']
Các giá trị duy nhất trong cột cap-shape là:
	 ['x' 'b' 's' 'f' 'k' 'c']
Các giá trị duy nhất trong cột cap-surface là:
	 ['s' 'y' 'f' 'g']
Các giá trị duy nhất trong cột cap-color là:
	 ['n' 'y' 'w' 'g' 'e' 'p' 'b' 'u' 'c' 'r']
Các giá trị duy nhất trong cột bruises là:
	 ['t' 'f']
Các giá trị duy nhất trong cột odor là:
	 ['p' 'a' 'l' 'n' 'f' 'c' 'y' 's' 'm']
Các giá trị duy nhất trong cột gill-attachment là:
	 ['f' 'a']
Các giá trị duy nhất trong cột gill-spacing là:
	 ['c' 'w']
Các giá trị duy nhất trong cột gill-size là:
	 ['n' 'b']
Các giá trị duy nhất trong cột gill-color là:
	 ['k' 'n' 'g' 'p' 'w' 'h' 'u' 'e' 'b' 'r' 'y' 'o']
Các giá trị duy nhất trong cột stalk-shape là:
	 ['e' 't']
Các giá trị duy nhất trong cột stalk-root là:
	 ['e' 'c' 'b' 'r' '?']
Các giá trị duy nhất trong cột stalk-surface-above-ring là:
	 ['s' 'f' 'k' 'y']
Các giá trị duy nhất trong cột stalk-surface-below-ring là:
	 ['s' 'f' 'y' 'k']
Các giá trị duy 

Lưu ý lúc này, 
- `target (y)` của chúng ta chính là cột đầu tiên trong dữ liệu
- `data (X)` là tất cả các cột còn lại,
- haha, `LOL`; hơn nữa tất cả các feature của chúng ta đều là các biến dạng `categorical`; chẳng hạn:

$\qquad \qquad \diamond$ `class (y)` hàm ý nấm này ăn được `e: edible` hay có độc `p : poisonus`

$\qquad \qquad \diamond$ `cap-shape`: lưu trữ hình dạng của nấm theo các kiểu khác nhau: `b: bell (hình chuông)`, `c: conial (hình nón)`, `x : convex (lồi)`, ...

$\qquad \qquad \diamond$ `cap-color`: lưu trữ màu sắc như `b: buff`, `y : yellow`, `w: white` và `n: brown`

$\qquad \qquad \diamond$  ....

Bây giờ, tiến hành dự đoán tương tự như bài toán trước; vẫn phải kiểm tra dữ liệu khuyết!

In [12]:
print(mushroom.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

Do tất cả các cột đều không có `missing-data` nên giờ ta sẽ tiến hành phân chia dữ liệu và dự đoán luôn

In [13]:
## Tách X và y
X = mushroom.drop(['class'], axis = 1)
y = mushroom['class']

# Encode categorical variables. Vì dữ liệu lúc này chứa quá nhiều biến dạng category nên ta phải thực hiện encoding cho chúng; ngăn cách với giá trị gốc bởi dấu _
X = pd.get_dummies(X, prefix_sep='_')

from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

## view X
X.head()

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


Train-test-split & predict

In [14]:
## train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    stratify = y, random_state = 0)

## predict model
clf_svm = SVC(kernel = 'poly', C = 10)   
clf_svm.fit(X_train, y_train)
print("train acc: ", accuracy_score(y_train, clf_svm.predict(X_train)))
print("train acc: ", accuracy_score(y_test, clf_svm.predict(X_test)))
print('Confusion matrix : \n', confusion_matrix(y_test, clf_svm.predict(X_test)))
print('Classification report : \n', classification_report(y_test, clf_svm.predict(X_test), digits = 5)) 

train acc:  1.0
train acc:  1.0
Confusion matrix : 
 [[1263    0]
 [   0 1175]]
Classification report : 
               precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000      1263
           1    1.00000   1.00000   1.00000      1175

    accuracy                        1.00000      2438
   macro avg    1.00000   1.00000   1.00000      2438
weighted avg    1.00000   1.00000   1.00000      2438



Như vậy, bài toán này dùng cùng mô hình `SVM` như bài toán 1 nhưng lại cho kết quả dự đoán chính xác tuyệt đối!

## Problem 3. Classification of Students' Academic Performance
Đầu tiên, tải dữ liệu rồi đánh giá sơ bộ

In [15]:
API_edu = pd.read_csv(r"/kaggle/input/xAPI-Edu-Data/xAPI-Edu-Data.csv")
API_edu.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M


Như vậy, 
- `target (y)` của bài toán này là cột cuối cùng `Class`
- `data (X)` chứa một cách lẫn lộn các biến liên tục như `raisedhands` (số lần giơ tay) và biến category như `Gender`.

Do đó, ta phải tạo ra dữ liệu `X` để train bằng cách gộp 2 nhóm dữ liệu `liên tục` với `encoding` của nhóm `category`

In [16]:
## tạo biến target
y = API_edu['Class']
## tạo data có dữ liệu là continuous
X_continuous = API_edu[['Discussion', 'AnnouncementsView', 'VisITedResources', 'raisedhands']]
## data với dữ liệu là category
X_category = API_edu.drop(['Class', 'Discussion', 'AnnouncementsView', 'VisITedResources', 'raisedhands'], axis = 1)
## encoding các data
X_category_encode = pd.get_dummies(X_category, prefix_sep='_')
y = LabelEncoder().fit_transform(y)
X = pd.concat([X_continuous, X_category_encode], axis = 1)

## train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    stratify = y, random_state = 0)

## predict model
clf_svm = SVC(kernel = 'poly', C = 10)   
clf_svm.fit(X_train, y_train)
print("train acc: ", accuracy_score(y_train, clf_svm.predict(X_train)))
print("train acc: ", accuracy_score(y_test, clf_svm.predict(X_test)))
print('Confusion matrix : \n', confusion_matrix(y_test, clf_svm.predict(X_test)))
print('Classification report : \n', classification_report(y_test, clf_svm.predict(X_test), digits = 5)) 

train acc:  0.7202380952380952
train acc:  0.6388888888888888
Confusion matrix : 
 [[19  3 21]
 [ 0 34  4]
 [ 9 15 39]]
Classification report : 
               precision    recall  f1-score   support

           0    0.67857   0.44186   0.53521        43
           1    0.65385   0.89474   0.75556        38
           2    0.60938   0.61905   0.61417        63

    accuracy                        0.63889       144
   macro avg    0.64726   0.65188   0.63498       144
weighted avg    0.64177   0.63889   0.62790       144



- Bài toán này có kết quả dự đoán độ chính xác trên tập train cao hơn nhiều so với trên tập test, đây là dấu hiệu của `overfit`
- Cả `train_acc` lẫn `test_acc` đều thấp trong mô hình này giải thích việc chúng ta chưa lựa chọn các `feature` tốt và phù hợp nhất cho mô hình.